<a href="https://colab.research.google.com/github/jatinparab/langchain-examples/blob/main/math_search_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken wikipedia langchain_openai langchainhub faiss-cpu

In [103]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor
from langchain import hub
from langchain_openai import ChatOpenAI, OpenAI
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [104]:
from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [30]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [84]:
llm = ChatOpenAI(model="gpt-4", temperature=0)

In [85]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [86]:
texts = [
    "George Washington was the first president of USA.",
    "He was born on Feb 22.",
    "He had 5 siblings.",
    "He died on Dec 14th.",
]
vectorstore = FAISS.from_texts(texts, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})
retriever_tool = create_retriever_tool(
    retriever,
    "george_washington_search",
    "Search for information about George Washington. For any questions about George Washington, you must use this tool!",
)
tools.append(retriever_tool)

In [97]:
prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
additional_instruction = "Don't stop until you've answered all questions: "

In [101]:
question = "Hi, how are you?"
# question = "Who is Jatin Parab? What is his profession?"
# question = "What are the 2 most populated states in India? What is the sum of population of these 2 states?"
# question = "Where is Codecademy's headquarter? Who was the first president of that country? How many siblings did the president have? What is the number of siblings divded by 5?"
result = agent_executor.invoke({"input": f"{additional_instruction}{question}", "chat_history": ""})
print(result['output'])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: Most populated states in India['1 (S1), Uttar Pradesh, 199,812,341, 16.51%, 20.2%, 155,317,278 · 2 (S2), Maharashtra, 112,374,333, 9.28%, 16%, 61,556,074 · 3 (S3), Bihar, 104,099,452, 8.6%, 25.4 ...', "Almost half of the country's population lives in the five highest populous states, namely, Uttar Pradesh, Bihar, Maharashtra, West Bengal and Madhya Pradesh.", 'The 5 most populated states in India are Uttar Pradesh, Maharashtra, Bihar, West Bengal, and Madhya Pradesh. Q2 ...', 'Uttar Pradesh is the most populous country subdivision in the world at 232 million people, while Sikkim, in the northeast of the country, is the ...', 'The most populated states in India are Uttar Pradesh, Maharashtra, and Bihar. These states boast significant populations, contributing to India ...', '1. Uttar Pradesh, 60,274,800 ; 2. Maharashtra, 32,002,500 ; 3. Bihar, 29,085,900 ; 4. West Bengal, 26,300,67